## Bayesian model comparison

# Intuition

**FIGURE PLACEHOLDER:** ![Bayesian model comparison](image_placeholder)

# Notation



## Bayes Factor (BF) (recall Lecture 8)
The **Bayes Factor (BF)** (sometimes denoted as $K$) is defined as the ratio of the marginal likelihoods of two models, taking into account the prior distributions over the models' parameters. It is calculated as:

$$
BF(M_1, M_2) := \frac{Pr(D|M_1)}{Pr(D|M_2)} = \frac{\int Pr(D|\theta_1, M_1) Pr(\theta_1|M_1) d\theta_1}{\int Pr(D|\theta_2, M_2) Pr(\theta_2|M_2) d\theta_2} = \frac{\frac{Pr(M_1|D)Pr(D)}{Pr(M_1)} }{\frac{Pr(M_2|D)Pr(D)}{Pr(M_2)}} = \frac{Pr(M_1|D)}{Pr(M_2|D)} \times \frac{Pr(M_2)}{Pr(M_1)}
$$

The Bayes Factor compares the marginal likelihoods of the models after considering the uncertainty in the parameters, **weighted by the prior beliefs of the model**.


### Compare $M_1$ and $M_2$ based on data $D$
When comparing two models $M_1$ and $M_2$, we calculate the **posterior odds** for each model. The **posterior odds** represent the ratio of the posterior probabilities for the two models. This can be written as:

$$
\frac{P(M_1|D)}{P(M_2|D)} = \frac{P(D|M_1)P(M_1)}{P(D|M_2)P(M_2)}
$$

Where:
- $P(M_1|D)$ is the **posterior** probability of model $M_1$ given the data $D$.
- $P(M_2|D)$ is the **posterior** probability of model $M_2$ given the data $D$.
- $\frac{P(D|M_1)}{P(D|M_2)}$ is the **bayes factor** of $M_1$ and $M_2$
- $P(M_1)$ and $P(M_2)$ are the **prior probabilities** of models $M_1$ and $M_2$, respectively.

This ratio simplifies to:

$$
\text{Posterior Odds} = \text{Prior Odds} \times \text{Likelihood Ratio (LR)}
$$

Where:
- **Prior Odds** = $\frac{P(M_1)}{P(M_2)}$ is the ratio of prior probabilities of the models.
- **Likelihood Ratio (LR)** = $\frac{P(D|M_1)}{P(D|M_2)}$ is the ratio of the likelihoods of the data under each model.

Thus, the **posterior odds** can be updated by multiplying the **prior odds** by the **likelihood ratio**.

# Example

**[???] Should we move the example in Lecture 8 here? Or what else ..**